In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn
!wget https://raw.githubusercontent.com/sajao/CrisisLex/master/releases/CrisisLexT26-v1.0.zip
!unzip CrisisLexT26-v1.0.zip
!pip install --quiet langdetect

--2019-09-15 06:46:25--  https://raw.githubusercontent.com/sajao/CrisisLex/master/releases/CrisisLexT26-v1.0.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4758309 (4.5M) [application/zip]
Saving to: ‘CrisisLexT26-v1.0.zip.2’

CrisisLexT26-v1.0.z 100%[===================>]   4.54M  --.-KB/s    in 0.1s    

2019-09-15 06:46:26 (39.2 MB/s) - ‘CrisisLexT26-v1.0.zip.2’ saved [4758309/4758309]

Archive:  CrisisLexT26-v1.0.zip
replace CrisisLexT26/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: CrisisLexT26/.DS_Store  
  inflating: __MACOSX/CrisisLexT26/._.DS_Store  
  inflating: CrisisLexT26/2012_Colorado_wildfires/2012_Colorado_wildfires-event_description.json  
  inflating: CrisisLexT26/2012_Colorado_wildfires/2012_Colorado_wildfires-tweetids_en

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import sys
from langdetect import detect
import json
from tqdm import tqdm

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
embed = hub.Module(module_url)
text_placeholder = tf.placeholder(tf.string, shape=[None])
embedding = embed(text_placeholder)
test_strings = ["OMG There is a huge typhoon in Miami. Save lives here: https://fdsjhfgsdeijg.com", "There is another report of a california fire.",
                "Hey look at my cute dog", "#TheFamilyManTrailer *When the topper ignores me during the exam* Me after some fake humble try:",
               "Now Post-Tropical (meaning technically no longer a hurricane) #Dorian making landfall in Canada this evening. Despite the name change, the system is still producing wind gusts up to 100mph! Hurricane Warnings in place along the southeast Canadian coast through the overnight."]
test_vectors = []
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  test_vectors = session.run(embedding, feed_dict={text_placeholder: test_strings})

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
test_pd = pd.read_csv('CrisisLexT26/2013_Spain_train_crash/2013_Spain_train_crash-tweets_labeled.csv')
test_pd.head()

,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness
0,359949548937945091,España busca exportar su tren de alta velocida...,Not labeled,Not labeled,Not related
1,360118977848016896,RT @elmundoes: URGENTE: Descarrila un tren de ...,Media,Other Useful Information,Related and informative
2,360120559088050176,RT @copecoruna: ÚLTIMA HORA: Descarrila un tre...,Media,Other Useful Information,Related and informative
3,360120953373593600,Ha descarrilado un tren en santiago de Compostela,Media,Other Useful Information,Related and informative
4,360121477636423683,RT @abc_es: #ACTUALIZACIÓN: Diez muertos tras ...,Media,Affected individuals,Related and informative


In [0]:
print(test_pd.keys())
print(test_pd.groupby([' Informativeness']).groups.keys())

Index(['Tweet ID', ' Tweet Text', ' Information Source', ' Information Type',
       ' Informativeness'],
      dtype='object')
dict_keys(['Not applicable', 'Not related', 'Related - but not informative', 'Related and informative'])


In [0]:
test_pd['lang'] = test_pd[' Tweet Text'].apply(detect)
test_pd.head()

,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,lang
0,359949548937945091,España busca exportar su tren de alta velocida...,Not labeled,Not labeled,Not related,es
1,360118977848016896,RT @elmundoes: URGENTE: Descarrila un tren de ...,Media,Other Useful Information,Related and informative,es
2,360120559088050176,RT @copecoruna: ÚLTIMA HORA: Descarrila un tre...,Media,Other Useful Information,Related and informative,es
3,360120953373593600,Ha descarrilado un tren en santiago de Compostela,Media,Other Useful Information,Related and informative,es
4,360121477636423683,RT @abc_es: #ACTUALIZACIÓN: Diez muertos tras ...,Media,Affected individuals,Related and informative,es


In [0]:
test_pd[test_pd['lang'] == 'en'].head()

,Tweet ID,Tweet Text,Information Source,Information Type,Informativeness,lang
6,360122056471355393,RT @NewsBreaker: BREAKING: Casualties reported...,Media,Other Useful Information,Related and informative,en
15,360127072837963776,@SteveInMadrid: BREAKING UPDATE 5 Exclusive Pi...,Media,Other Useful Information,Related and informative,en
18,360127764927479808,Train Crash In Spain: Several Deaths Reported ...,Media,Other Useful Information,Related and informative,en
24,360129643967299584,Spain high-speed train crashes near Santiago d...,Media,Other Useful Information,Related and informative,en
34,360134064734347264,RT @NewsBreaker: BREAKING PIC: 4 dead after tr...,Media,Affected individuals,Related and informative,en


In [0]:
ignore_types = ['Collapse', 'Meteorite', 'Haze']
type_maps = {'Floods': 'Floods',
            'Typhoon': 'Typhoon',
            'Shootings': 'Terrorism', 'Bombings': 'Terrorism',
            'Wildfire': 'Fire', 'Fire': 'Fire',
            'Derailment': 'Crash', 'Crash': 'Crash',
            'Explosion': 'Explosion',
            'Earthquake': 'Earthquake'}
all_disasters = [os.path.join('CrisisLexT26', f) for f in os.listdir('CrisisLexT26')]
# print(all_disasters)
type_dict = dict()
category_dict = dict()

tweets = []
tweets_type = []
related = []


for all_d in tqdm(all_disasters):
  try:
    json_file = [os.path.join(all_d, f) for f in os.listdir(all_d) if f.endswith("json")][0]
    csv_file = [os.path.join(all_d, f) for f in os.listdir(all_d) if f.endswith("-tweets_labeled.csv")][0]
    json_dict = json.load(open(json_file, "r"))
    this_type = json_dict['categorization']['type']
    if this_type in ignore_types:
      continue
    this_new_type = type_maps[this_type]
    this_df = pd.read_csv(csv_file)
    print(len(this_df))
    this_df['lang'] = this_df[' Tweet Text'].apply(detect)
    this_df = this_df[this_df['lang'] == 'en']
    this_tweet_list = this_df[' Tweet Text'].tolist()
    this_df['related'] = this_df[' Informativeness'].apply(lambda x: int(x in ['Related - but not informative', 'Related and informative']))
    this_related_list = this_df['related'].tolist()
    tweets.extend(this_tweet_list)
    related.extend(this_related_list)
    tweets_type.extend([this_new_type for _ in this_related_list])
    # type_dict[json_dict['categorization']['type']] = type_dict.get(json_dict['categorization']['type'], 0) + 1
    # category_dict[json_dict['categorization']['category']] = category_dict.get(json_dict['categorization']['category'], 0) + 1
  except Exception as e:
    print(e)
    continue


  0%|          | 0/28 [00:00<?, ?it/s]

1200



  4%|▎         | 1/28 [00:06<03:02,  6.75s/it]

1000



  7%|▋         | 2/28 [00:13<02:51,  6.61s/it]

...


 93%|█████████▎| 26/28 [02:19<00:13,  6.97s/it]

1000



 96%|█████████▋| 27/28 [02:24<00:06,  6.61s/it]

1048



100%|██████████| 28/28 [02:30<00:00,  6.33s/it]


In [0]:
over_df = pd.DataFrame({'Tweet': tweets, 'Type': tweets_type, 'Related': related})
print(len(tweets))
over_df.groupby('Type').count()

17120


,Tweet,Related
Type,,
Crash,2974,2974
Earthquake,1922,1922
Explosion,1018,1018
Fire,2738,2738
Floods,4803,4803
Terrorism,1913,1913
Typhoon,1752,1752


In [0]:
over_df.groupby('Related').count()

,Tweet,Type
Related,,
0,1703,1703
1,15417,15417


In [0]:
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  over_df['Vectors'] = session.run(embed(over_df['Tweet'].tolist())).tolist()
over_df.head(10)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


,Tweet,Type,Related,Vectors
0,RT @Jack4Ward: Get in on the fun every Thursda...,Fire,0,"[0.018871961161494255, 0.05218875780701637, -0..."
1,Welcome to our newest STUDENTathlete- Reagan B...,Fire,0,"[0.024413956329226494, 0.035585999488830566, -..."
2,Denver Post: #Colorado governor signs bill cre...,Fire,0,"[0.04952438175678253, -0.04628261923789978, -0..."
3,Pretty sure I'm going to live in Manitou Sprin...,Fire,0,"[0.0073580024763941765, 0.036944229155778885, ..."
4,Flying over the beautiful snow capped #mountai...,Fire,0,"[0.02702946774661541, -0.009432152844965458, -..."
5,It isn't a beach but just as great #colorado #...,Fire,0,"[-0.045321449637413025, 0.06808455288410187, -..."
6,#colorado. Told you its #amazing http://t.co/6...,Fire,0,"[0.031008010730147362, -0.008062882348895073, ..."
7,RT @xRedRoverx: #Colorado #news #crime 3 bodie...,Fire,0,"[0.029940232634544373, -0.07236875593662262, -..."
8,checkmate payday loans colorado springs http:/...,Fire,0,"[0.06465433537960052, 0.045164112001657486, -0..."
9,"Free #HTC #Wildfire S, 250 minutes, 5,000 text...",Fire,0,"[0.040880363434553146, 0.021573971956968307, -..."


In [0]:
idx2text = list(over_df.groupby(['Type']).groups.keys()) + ['None']
text2idx = dict((x.strip(), i) for (i, x) in enumerate(idx2text))
print(idx2text)
print(text2idx)

['Crash', 'Earthquake', 'Explosion', 'Fire', 'Floods', 'Terrorism', 'Typhoon', 'None']
{'Crash': 0, 'Earthquake': 1, 'Explosion': 2, 'Fire': 3, 'Floods': 4, 'Terrorism': 5, 'Typhoon': 6, 'None': 7}


In [0]:
over_df['Type_INT'] = over_df['Type'].apply(lambda x: text2idx[x])
over_df.head()

,Tweet,Type,Related,Vectors,Type_INT
0,RT @Jack4Ward: Get in on the fun every Thursda...,Fire,0,"[0.018871961161494255, 0.05218875780701637, -0...",3
1,Welcome to our newest STUDENTathlete- Reagan B...,Fire,0,"[0.024413956329226494, 0.035585999488830566, -...",3
2,Denver Post: #Colorado governor signs bill cre...,Fire,0,"[0.04952438175678253, -0.04628261923789978, -0...",3
3,Pretty sure I'm going to live in Manitou Sprin...,Fire,0,"[0.0073580024763941765, 0.036944229155778885, ...",3
4,Flying over the beautiful snow capped #mountai...,Fire,0,"[0.02702946774661541, -0.009432152844965458, -...",3


In [0]:
over_df.to_csv('vectored.csv')

In [0]:
zipped_ds = list(zip(over_df['Related'].tolist(), over_df['Vectors'].tolist(), over_df['Type_INT'].tolist()))
new_zipped_ds = []
posw = np.zeros([8])
for r, vc, ti in zipped_ds:
  if r == 1:
    posw[ti] += 1
    new_zipped_ds.append((1, vc, ti))
  else:
    posw[7] += 1
    new_zipped_ds.append((0, vc, 7))
zipped_ds = new_zipped_ds
import random
random.shuffle(zipped_ds)
train_count = int(0.9*len(zipped_ds))
train_set = zipped_ds[:train_count]
test_set = zipped_ds[train_count:]

def gelu_fast(_x):
  return 0.5 * _x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (_x + 0.044715 * tf.pow(_x, 3))))
print(len(train_set))
print(len(test_set))
print(posw)

15408
1712
[2776. 1596.  946. 2267. 4359. 1816. 1657. 1703.]


In [0]:
tf.reset_default_graph()
with tf.Session() as session:
  batch_size = 16
  y_placeholder = tf.placeholder(tf.int32, shape=[batch_size])
  type_placeholder = tf.placeholder(tf.int32, shape=[batch_size])
  tp_onehot = tf.one_hot(type_placeholder, 8, dtype=tf.float32)
  vector_placeholder = tf.placeholder(tf.float32, shape=[batch_size, 512])
  dropout_val = tf.placeholder(tf.float32, shape=[])
#   with tf.variable_scope("wtf", reuse=tf.AUTO_REUSE) as scope:
#     trainable_embeddings = tf.get_variable("trainable_embedding", shape=[7, 32],
#           initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float32)

#     x = tf.nn.embedding_lookup(trainable_embeddings, type_placeholder) # (batch_size, 1, 32)
#   print(x)
  # x = tf.reshape(x, [batch_size, 32])
  # x = tf.concat([vector_placeholder, x], axis=-1)
  x = vector_placeholder

  x = tf.layers.dense(x, 256, activation=gelu_fast, trainable=True, name="l1",
      reuse=tf.AUTO_REUSE)
  x = tf.layers.dense(x, 64, activation=gelu_fast, trainable=True, name="l2",
      reuse=tf.AUTO_REUSE)
  x = tf.nn.dropout(x, dropout_val)
  print(x)
  logits = tf.layers.dense(x, 8, activation=None, trainable=True, name="fc",
      reuse=tf.AUTO_REUSE)
  probs = tf.nn.softmax(logits, axis=-1)
  print(logits)

  print(y_placeholder)
  cost = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(labels=tp_onehot, logits=logits, pos_weight=17121/posw)) # tf.nn.sparse_softmax_cross_entropy_with_logits(labels=type_placeholder, logits=logits))
  predictionsss = tf.argmax(logits, axis=-1)
  conf_matrix = tf.math.confusion_matrix(type_placeholder, predictionsss, num_classes=8)
  acc, acc_op = tf.metrics.accuracy(type_placeholder,
                                    predictionsss)
  
  cel = tf.summary.scalar('cross-entropy-loss', cost)
  acc_s = tf.summary.scalar('avg_acc', acc)
  summary = tf.summary.merge([cel, acc_s])
  
  print(tf.trainable_variables())

  optimizer = tf.train.AdamOptimizer(0.001)
  gradsvars = optimizer.compute_gradients(cost, var_list=tf.trainable_variables())
  grads, _ = tf.clip_by_global_norm([g for g, v in gradsvars], 50)
  optimize_fn = optimizer.apply_gradients(zip(grads, tf.trainable_variables()))
  
  running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES)
  running_vars_initializer = tf.variables_initializer(var_list=running_vars)
  
  saver = tf.train.Saver(max_to_keep=5)

  session.run(tf.global_variables_initializer())
  
  for ep in range(7):
    train_count = len(train_set) // batch_size
    test_count = len(test_set) // batch_size
    session.run(running_vars_initializer)
    total_cost = 0
    total_conf_matrix = np.zeros([8, 8], dtype=np.int32)
    for i in range(train_count):
      xxx = train_set[i*batch_size: (i+1)*batch_size]
      y_output, input_vectors, input_type = zip(*xxx)
      c, _, _, cf = session.run([cost, optimize_fn, acc_op, conf_matrix],
                            feed_dict={y_placeholder: y_output,
                                       vector_placeholder: input_vectors,
                                       type_placeholder: input_type,
                                       dropout_val: 0.5})
      total_conf_matrix += cf
      total_cost += c
      # if tc % (train_count//2) == 0:
      #   print(c, session.run(acc))
    print("---TRAIN---")
    print(total_cost/train_count, session.run(acc))
    saver.save(session, "DisasterNet", global_step=ep)
    print(total_conf_matrix)
    
    session.run(running_vars_initializer)
    total_cost = 0
    total_conf_matrix = np.zeros([8, 8], dtype=np.int32)
    for i in range(test_count):
      xxx = test_set[i*batch_size: (i+1)*batch_size]
      y_output, input_vectors, input_type = zip(*xxx)
      c, _, cf = session.run([cost, acc_op, conf_matrix],
                            feed_dict={y_placeholder: y_output,
                                       vector_placeholder: input_vectors,
                                       type_placeholder: input_type,
                                       dropout_val: 1.0})
      total_conf_matrix += cf
      total_cost += c
      # if tc % (test_count//2) == 0:
      #   print(c, session.run(acc))
    print("---TEST---")
    print(total_cost/test_count, session.run(acc))
    print(total_conf_matrix)

    for tv in test_vectors:
      ppp = session.run(probs, feed_dict={vector_placeholder: [tv]*batch_size, dropout_val: 1.0})[0]
      pp_val = np.argmax(ppp)
      print(idx2text[pp_val], ppp[pp_val])
    

Tensor("dropout/mul_1:0", shape=(16, 64), dtype=float32)
Tensor("fc/BiasAdd:0", shape=(16, 8), dtype=float32)
Tensor("Placeholder:0", shape=(16,), dtype=int32)
[<tf.Variable 'l1/kernel:0' shape=(512, 256) dtype=float32_ref>, <tf.Variable 'l1/bias:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'l2/kernel:0' shape=(256, 64) dtype=float32_ref>, <tf.Variable 'l2/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'fc/kernel:0' shape=(64, 8) dtype=float32_ref>, <tf.Variable 'fc/bias:0' shape=(8,) dtype=float32_ref>]
---TRAIN---
0.6657819662200575 0.6725727
[[2142   15   54   32  104   55   48   38]
 [  10 1065   63   13   55   20  131   89]
 [  13   23  571   62   43   78   19   43]
 [  72   30  162 1261  311   63   37   96]
 [ 136  103  171  240 2502   68  462  221]
 [  76   16   96   23   23 1316   27   52]
 [  24  139   39   33  276   25  908   61]
 [ 108  193   68  146  201  122  117  598]]
---TEST---
0.4774032938424672 0.75
[[258   2   4   6  10   2   3   3]
 [  0 127   2   1   5   